In [1]:
%load_ext autoreload
%autoreload 2

import json
import random
import torch
import numpy as np
from PIL import Image, ImageDraw

import sys
sys.path.append("../detr")
import util.misc as utils
from eval import visualize, visualize_2

from main import get_transform, get_model

In [2]:
cmd_args = {
    "data_type": "structure",
    "config_file": "./structure_config.json",
    "model_load_path": "../models/pubtables1m_structure_detr_r18.pth",
    "table_words_dir": "../data/Own-Samples",
}
config_args = json.load(open(cmd_args["config_file"], "rb"))
config_args["device"] = "cpu"

for key, value in cmd_args.items():
    if not key in config_args or not value is None:
        config_args[key] = value
args = type("Args", (object,), config_args)
#print(args.__dict__)

In [3]:
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = args.device
model, criterion, postprocessors = get_model(args, device)

loading model from checkpoint


In [4]:
model.eval();

In [5]:
criterion.eval();

In [6]:
postprocessors

{'bbox': PostProcess()}

In [7]:
target = {
    "img_path": "../data/Own-Samples/Picture4.jpg",
    "boxes":[],
}

img_path = target["img_path"]
img = Image.open(img_path).convert("RGB")
print("Image size:", img.size)

transforms =  get_transform(args.data_type, "val")
img_tensor, target = transforms(img, target)
print("Tensor size:", img_tensor.size())

outputs = model(img_tensor.unsqueeze(0))
#print(outputs.keys())

pred_logits = outputs["pred_logits"][0]
pred_boxes = outputs["pred_boxes"][0]
#visualize(args, target, pred_logits, pred_boxes)


Image size: (4900, 2110)
Tensor size: torch.Size([3, 431, 1000])


In [8]:
_, pred_cells, _ = visualize_2(args, target, pred_logits, pred_boxes)

In [9]:
pred_cells[0]

{'bbox': [293.0, 242.0, 1844.0, 302.0],
 'column_nums': [0],
 'row_nums': [0],
 'header': True,
 'subcell': False,
 'subheader': False,
 'cell_text': '',
 'spans': []}

In [10]:
print(len(pred_cells))
print(set([d['column_nums'][0] for d in pred_cells]))
print(set([d['row_nums'][0] for d in pred_cells]))
print(set([d['header'] for d in pred_cells]))
print(set([d.get('subcell') for d in pred_cells]))
print(set([d['subheader'] for d in pred_cells]))
print(set([d['cell_text'] for d in pred_cells]))

60
{0, 1, 2, 3, 4, 5, 6}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{False, True}
{False, None}
{False}
{'', '(8)', '(10)', '2019', '307', '304', '316', '320', 'Share-based payments', '56', 'Operating profit', '71', '260', '(7)', '391', '284', 'Insurance £m', '243', '(32)', '403', 'Trading EBITDA', '66', '337', 'Roadside £m', '(12)', 'Amortisation and depreciation', 'Exceptional operating items', '(1)', '(6)', '(67)', '61', 'Pension service charge adjustment', '(63)', 'Group £m', '(70)', '2018', 'Operating profit before exceptional items', '(60)', '(2)'}


In [13]:
for d in pred_cells:
    if d['cell_text'] in ["2018", "2019", "Roadside £m", "Share-based payments"]:
        print(d['cell_text'])
        print(d)

Share-based payments
{'bbox': [293.0, 840.0, 1844.0, 920.0], 'column_nums': [0], 'row_nums': [4], 'header': False, 'subcell': False, 'subheader': False, 'cell_text': 'Share-based payments', 'spans': [{'bbox': [296, 841, 747, 904], 'text': 'Share-based', 'flags': 0, 'span_num': 358, 'line_num': 0, 'block_num': 0}, {'bbox': [781, 849, 1140, 920], 'text': 'payments', 'flags': 0, 'span_num': 359, 'line_num': 0, 'block_num': 0}]}
2018
{'bbox': [2049.0, 242.0, 3231.0, 302.0], 'column_nums': [1, 2, 3], 'row_nums': [0], 'header': True, 'subheader': False, 'cell_text': '2018', 'spans': [{'bbox': [2518, 242, 2695, 302], 'text': '2018', 'flags': 0, 'span_num': 384, 'line_num': 0, 'block_num': 0}]}
2019
{'bbox': [3424.0, 242.0, 4605.0, 302.0], 'column_nums': [4, 5, 6], 'row_nums': [0], 'header': True, 'subheader': False, 'cell_text': '2019', 'spans': [{'bbox': [3891, 242, 4069, 302], 'text': '2019', 'flags': 0, 'span_num': 407, 'line_num': 0, 'block_num': 0}]}
Roadside £m
{'bbox': [2049.0, 358.0, 

In [12]:
#for d in pred_cells:
#    if d.get('spans'):
#        print(d['cell_text'])
#        print(d)